In [1]:
# steps
# 1. Design model (input, output, forward pass)
# 2. Construct loss and optimizer

# 3. Training Loop
    # - forward pass -> compute prediction
    # - backward pass -> gradiants
    # - update weights

In [3]:
import torch
import torch.nn as nn
import numpy as np 
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [17]:
# Steps
# 0. Prepare dataset
bc = datasets.load_breast_cancer()

X,y = bc.data, bc.target

n_sample, n_features = X.shape
X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state = 42)

sc = StandardScaler() # mean 0 and variance 1

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#convert to tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

#reshape y tensors

y_train = y_train.view(y_train.shape[0], 1) #here 1 means all the y in 1 column
y_test = y_test.view(y_test.shape[0], 1)

# 1. Design model
#f = wx + b

class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    
model = LogisticRegression(n_features)
        
# 2. Loss and optimizer
lrrr = .01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lrrr)

# 3. Training Loop

num_epochs = 100

for epoch in range(num_epochs):
    # forward pass and loss calculation
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    #backward pass
    loss.backward()
    #updates
    optimizer.step()
    
    #empty the gradients
    optimizer.zero_grad()
    
    if(epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss={loss.item():.4f}')
        
with torch.no_grad():
    y_predicted = model(X_test)
    y_class = y_predicted.round() # more than .5 will be considered as class 1
    acc = y_class.eq(y_test).sum() / float(y_test.shape[0]) # for every correct guess +1
    print(f'accuracy = {acc:.4f}')
    



epoch: 10, loss=0.5345
epoch: 20, loss=0.4628
epoch: 30, loss=0.4134
epoch: 40, loss=0.3770
epoch: 50, loss=0.3488
epoch: 60, loss=0.3261
epoch: 70, loss=0.3074
epoch: 80, loss=0.2917
epoch: 90, loss=0.2782
epoch: 100, loss=0.2665
accuracy = 0.9649
